# Этап 1. Распознавание городов

Следующая часть кода позволяет из неструктурированного адреса выделить название города. Результаты обработки помещаются в новые столбцы "city_from" и "city_to"

In [214]:
# Импортирование библиотек
import pandas as pd
import numpy as np
import re

In [215]:
# Загрузка файла с данными
file = 'data_example.xlsx'
data_tab = pd.read_excel(file)

In [216]:
# Загрузка файла с городами
spisok_gor_f = 'spisok_gorodov.xlsx'
spisok_gor = pd.read_excel(spisok_gor_f)
# Сортировка городов по количеству символов в названии
spisok_gor.loc[:, 'num_sim']=spisok_gor.loc[:, 'Город'].str.len()
spisok_gor=spisok_gor.sort_values(by=['num_sim'])

In [217]:
# Загрузка файла с исключениями для обработки опечаток и различных наименований одного города
diction=pd.read_excel('exceptions.xlsx', index_col=1)
diction.head()
diction=diction['city'].to_dict()
dict_val=list(diction.keys())

In [218]:
spisok_gor.head()

,Unnamed: 0,Индекс,Регион,Тип района,Район,Тип города,Город,Тип н/п,Н/п,Код КЛАДР,...,Код ОКАТО,Код ОКТМО,Код ИФНС,Часовой пояс,Широта,Долгота,Федеральный округ,Население,дубликат,num_sim
1134,NaN,NaN,Свердловская область,NaN,NaN,г,Реж,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
796,808.0,694490.0,Сахалинская область,р-н,Охинский,г,Оха,NaN,NaN,6.500900e+12,...,6.442500e+10,6.473600e+10,6517.0,UTC+11,53.586824,142.941221,Дальневосточный,23007,NaN,3
413,417.0,663920.0,Красноярский край,р-н,Уярский,г,Уяр,NaN,NaN,2.404100e+12,...,4.257501e+09,4.657101e+06,2448.0,UTC+7,55.813170,94.328215,Сибирский,12666,NaN,3
354,358.0,157000.0,Костромская область,р-н,Буйский,г,Буй,NaN,NaN,4.400300e+12,...,3.440500e+10,3.470500e+07,4437.0,UTC+3,58.473445,41.530675,Центральный,25763,NaN,3
689,699.0,618120.0,Пермский край,р-н,Осинский,г,Оса,NaN,NaN,5.901600e+12,...,5.724050e+10,5.764010e+10,5900.0,UTC+5,57.288999,55.468898,Приволжский,22420,NaN,3


In [219]:
data_tab.head()

,ID,Город отгрузки,Город выгрузки,Количество грузовых мест,"Габариты грузовых мест (Д*Ш*В), м","Общий вес груза, кг",ТИП ТС
0,1,"АО ""ЕЕЕ""\nАдрес: Нижегородская обл, Дзержинск ...",Грузополучатель: Общество с ограниченной ответ...,1,"0,8х0,5х0,8","5,67 тн",до 0.5 тн
1,2,"""Торговое представительство АО """"ННН"""" 467007,...","ООО ""Газпромнефть-Хантос"" ИНН 8618001111/КПП 9...",2,"200*200*570, 900*870*650",750 кг.,"от 3,1 до 5"
2,3,"АО ""УУУ"", фактический адрес склада: 133354 г.М...","623751, Свердловская область., Реж г., Калинин...",1,"1.2*0.8*0,66",16 000 кг,негабарит до 30 тн
3,4,ЮПРМ,"""ООО """"УУУ"""", Пермский край, город Лысьва, ули...",1,36*19*14 см,20 ТН,"от 10,1 до 20 тн"
4,5,"""Грузополучатель: ООО «ЕЕЕ» база «аааа», РФ, ...","Челябинск, ул Рабочая 33в Время...",3,1. Грузовое место 1/3: \nДхШхВ=3280х1660х1320 ...,20 ТН,5


In [220]:
# Список с вариантами написания города для обработки строк, где город явно указан (повышает точность обработки)
list_designation=['г.', ' город', 'г ']

Так как специфика города отгрузки и города выгрузки разная, код был написан с учетом особенностей отдельно для города выгрузки и города отгрузки

In [221]:
# Только для города отгрузки
# Создание пустого столбца в таблице
data_tab['city_from']=np.NaN
# Обработка в несколько циклов: для строк, где город явно указан, для исключений, для строк, где нет указания города
for i in range(data_tab.shape[0]):
    value=str(data_tab.loc[i, 'Город отгрузки '])
    for s in range(spisok_gor.shape[0]):
        if str(spisok_gor.loc[s, 'Город'])+' г' in value:
            value=value.replace(str(spisok_gor.loc[s, 'Город'])+' г', 'г.'+str(spisok_gor.loc[s, 'Город']))
    if '.' or ',' in value:
        value=value.replace('ё', 'е')
        value=value.split(',')
        value=[value[o].strip() for o in range(len(value))]
        value=', '.join(value)
        for k in range(len(list_designation)):
            if list_designation[k] in value:
                index1=value.find(list_designation[k])
                len_designation=len(list_designation[k])
                sum_find=index1+len_designation
                index2=value.find(' ', sum_find+1)
                data_tab.loc[i, 'city_from'] = value[sum_find:index2+1].replace(',', '').replace('.', '').strip()
    value2=value.strip()
    if spisok_gor['Город'].isin([value2]).any():
        data_tab.loc[i, 'city_from'] = str(value2)
for i in range(data_tab.shape[0]):
    if spisok_gor['Город'].isin([data_tab.loc[i, 'city_from']]).any()==False:
        data_tab.loc[i, 'city_from'] = np.NaN
for i in range(data_tab.shape[0]):
    if pd.isnull(data_tab.loc[i, 'city_from']) and pd.notnull(data_tab.loc[i, 'Город отгрузки ']):
        value=str(data_tab.loc[i, 'Город отгрузки ']).replace(',', '')
        for k in range(spisok_gor.shape[0]):
            m=str(spisok_gor.loc[k, 'Город'])+' '
            n=str(spisok_gor.loc[k, 'Город'])+','
            if n in value:
                data_tab.loc[i, 'city_from'] = spisok_gor.loc[k, 'Город']
            if m in value:
                data_tab.loc[i, 'city_from'] = spisok_gor.loc[k, 'Город']
for i in range(data_tab.shape[0]):
    if pd.isnull(data_tab.loc[i, 'city_from']) and pd.notnull(data_tab.loc[i, 'Город отгрузки ']):
        value=str(data_tab.loc[i, 'Город отгрузки ']).replace(',', '')
        for k in range(len(dict_val)):
            if dict_val[k] in value:
                data_tab.loc[i, 'city_from'] = diction[dict_val[k]]  
for i in range(data_tab.shape[0]):
    if pd.isnull(data_tab.loc[i, 'city_from']) and pd.notnull(data_tab.loc[i, 'Город отгрузки ']):
        value=str(data_tab.loc[i, 'Город отгрузки ']).replace(',', ' ')+' '
        for k in range(spisok_gor.shape[0]):
            if str(spisok_gor.loc[k, 'Город'])+' ' in value:
                data_tab.loc[i, 'city_from'] = spisok_gor.loc[k, 'Город']
# Вывод доли определенных городов от общего количества
a = len(data_tab[data_tab['city_from'].isnull()==False]['city_from'])
b = len(data_tab[data_tab['Город отгрузки '].isnull()==False]['Город отгрузки '])
print(a/b)

1.0


In [222]:
# Только для города выгрузки
# Создание пустого столбца в таблице
data_tab['city_to']=np.NaN
# Обработка в несколько циклов: для строк, где город явно указан, для исключений, для строк, где нет указания города
for i in range(data_tab.shape[0]):
    value=str(data_tab.loc[i, 'Город выгрузки '])
    for s in range(spisok_gor.shape[0]):
        if str(spisok_gor.loc[s, 'Город'])+' г' in value:
            value=value.replace(str(spisok_gor.loc[s, 'Город'])+' г', 'г.'+str(spisok_gor.loc[s, 'Город']))
    if '.' or ',' in value:
        value=value.replace('ё', 'е')
        value=value.split(',')
        value=[value[o].strip() for o in range(len(value))]
        value=', '.join(value)
        for k in range(len(list_designation)):
            if list_designation[k] in value:
                index1=value.find(list_designation[k])
                len_designation=len(list_designation[k])
                sum_find=index1+len_designation
                index2=value.find(' ', sum_find+1)
                data_tab.loc[i, 'city_to'] = value[sum_find:index2+1].replace(',', '').replace('.', '').strip()
    value2=value.strip()
    if spisok_gor['Город'].isin([value2]).any():
        data_tab.loc[i, 'city_to'] = str(value2)
for i in range(data_tab.shape[0]):
    if spisok_gor['Город'].isin([data_tab.loc[i, 'city_to']]).any()==False:
        data_tab.loc[i, 'city_to'] = np.NaN
for i in range(data_tab.shape[0]):
    if pd.isnull(data_tab.loc[i, 'city_to']) and pd.notnull(data_tab.loc[i, 'Город выгрузки ']):
        value=str(data_tab.loc[i, 'Город выгрузки ']).replace(',', '')
        for k in range(spisok_gor.shape[0]):
            m=str(spisok_gor.loc[k, 'Город'])+' '
            n=str(spisok_gor.loc[k, 'Город'])+','
            if n in value:
                data_tab.loc[i, 'city_to'] = spisok_gor.loc[k, 'Город']
            if m in value:
                data_tab.loc[i, 'city_to'] = spisok_gor.loc[k, 'Город']
for i in range(data_tab.shape[0]):
    if pd.isnull(data_tab.loc[i, 'city_to']) and pd.notnull(data_tab.loc[i, 'Город выгрузки ']):
        value=str(data_tab.loc[i, 'Город выгрузки ']).replace(',', '')
        for k in range(len(dict_val)):
            if dict_val[k] in value:
                data_tab.loc[i, 'city_to'] = diction[dict_val[k]]  
for i in range(data_tab.shape[0]):
    if pd.isnull(data_tab.loc[i, 'city_to']) and pd.notnull(data_tab.loc[i, 'Город выгрузки ']):
        value=str(data_tab.loc[i, 'Город выгрузки ']).replace(',', ' ')+' '
        for k in range(spisok_gor.shape[0]):
            if str(spisok_gor.loc[k, 'Город'])+' ' in value:
                data_tab.loc[i, 'city_to'] = spisok_gor.loc[k, 'Город']      
# Вывод доли определенных городов от общего количества
a = len(data_tab[data_tab['city_to'].isnull()==False]['city_to'])
b = len(data_tab[data_tab['Город выгрузки '].isnull()==False]['Город выгрузки '])
print(a/b)

1.0


In [223]:
print(data_tab)

   ID                                    Город отгрузки   \
0   1  АО "ЕЕЕ"\nАдрес: Нижегородская обл, Дзержинск ...   
1   2  "Торговое представительство АО ""ННН"" 467007,...   
2   3  АО "УУУ", фактический адрес склада: 133354 г.М...   
3   4                                               ЮПРМ   
4   5  "Грузополучатель: ООО «ЕЕЕ»  база «аааа», РФ, ...   

                                     Город выгрузки   \
0  Грузополучатель: Общество с ограниченной ответ...   
1  ООО "Газпромнефть-Хантос" ИНН 8618001111/КПП 9...   
2  623751, Свердловская область., Реж г., Калинин...   
3  "ООО ""УУУ"", Пермский край, город Лысьва, ули...   
4  Челябинск, ул Рабочая 33в                Время...   

   Количество грузовых мест  \
0                         1   
1                         2   
2                         1   
3                         1   
4                         3   

                   Габариты грузовых мест (Д*Ш*В), м Общий вес груза, кг  \
0                                      

In [224]:
# Соединение с таблицей всех городов с координатами, регионами
spisok_gor_inf=spisok_gor[['Город', 'Регион', 'Широта', 'Долгота']]
joined_df1=pd.merge(data_tab, spisok_gor_inf, left_on='city_from', right_on='Город', how='left')
joined_df2=pd.merge(joined_df1, spisok_gor_inf, left_on='city_to', right_on='Город', how='left')
joined_df2=joined_df2.drop(labels=['city_from', 'city_to'], axis=1)

In [225]:
print(joined_df2)

   ID                                    Город отгрузки   \
0   1  АО "ЕЕЕ"\nАдрес: Нижегородская обл, Дзержинск ...   
1   2  "Торговое представительство АО ""ННН"" 467007,...   
2   3  АО "УУУ", фактический адрес склада: 133354 г.М...   
3   4                                               ЮПРМ   
4   5  "Грузополучатель: ООО «ЕЕЕ»  база «аааа», РФ, ...   

                                     Город выгрузки   \
0  Грузополучатель: Общество с ограниченной ответ...   
1  ООО "Газпромнефть-Хантос" ИНН 8618001111/КПП 9...   
2  623751, Свердловская область., Реж г., Калинин...   
3  "ООО ""УУУ"", Пермский край, город Лысьва, ули...   
4  Челябинск, ул Рабочая 33в                Время...   

   Количество грузовых мест  \
0                         1   
1                         2   
2                         1   
3                         1   
4                         3   

                   Габариты грузовых мест (Д*Ш*В), м Общий вес груза, кг  \
0                                      

In [226]:
data_tab=joined_df2

# Этап 2. Распознавание типа транспортного средства

Следующая часть кода позволяет определить тип транспортного средства исходя из списка значений

In [227]:
# Создание пустых столбцов
data_tab['TS type']=np.nan
data_tab['weight']=np.nan

In [228]:
data_tab.head()

,ID,Город отгрузки,Город выгрузки,Количество грузовых мест,"Габариты грузовых мест (Д*Ш*В), м","Общий вес груза, кг",ТИП ТС,Город_x,Регион_x,Широта_x,Долгота_x,Город_y,Регион_y,Широта_y,Долгота_y,TS type,weight
0,1,"АО ""ЕЕЕ""\nАдрес: Нижегородская обл, Дзержинск ...",Грузополучатель: Общество с ограниченной ответ...,1,"0,8х0,5х0,8","5,67 тн",до 0.5 тн,Дзержинск,Нижегородская область,56.237644,43.459936,Мегион,Ханты-Мансийский Автономный округ - Югра,61.031895,76.102477,NaN,NaN
1,2,"""Торговое представительство АО """"ННН"""" 467007,...","ООО ""Газпромнефть-Хантос"" ИНН 8618001111/КПП 9...",2,"200*200*570, 900*870*650",750 кг.,"от 3,1 до 5",Ульяновск,Ульяновская область,54.308067,48.374872,Приобское м/р ЮЛТ,Ханты-Мансийский Автономный округ - Югра,NaN,NaN,NaN,NaN
2,3,"АО ""УУУ"", фактический адрес склада: 133354 г.М...","623751, Свердловская область., Реж г., Калинин...",1,"1.2*0.8*0,66",16 000 кг,негабарит до 30 тн,Москва,Москва,55.753879,37.620373,Реж,Свердловская область,NaN,NaN,NaN,NaN
3,4,ЮПРМ,"""ООО """"УУУ"""", Пермский край, город Лысьва, ули...",1,36*19*14 см,20 ТН,"от 10,1 до 20 тн",Приобское м/р ЮЛТ,Ханты-Мансийский Автономный округ - Югра,NaN,NaN,Лысьва,Пермский край,58.099599,57.808573,NaN,NaN
4,5,"""Грузополучатель: ООО «ЕЕЕ» база «аааа», РФ, ...","Челябинск, ул Рабочая 33в Время...",3,1. Грузовое место 1/3: \nДхШхВ=3280х1660х1320 ...,20 ТН,5,Лабытнанги,Ямало-Ненецкий автономный округ,66.659238,66.388263,Челябинск,Челябинская область,55.160366,61.400786,NaN,NaN


In [229]:
data_tab.loc[data_tab['ТИП ТС '].isin([1, '1', 'до 0,5 тн ', 'до 0,5 тн', 'до 0,5  тн', 'до 0.5 тн ', 'до 0.5 тн', 'до 0.5 тн ', 'до 0,5тн ', 'до 0,5  тн ', 'до 0,5 ']), 'TS type']=1

In [230]:
data_tab.loc[data_tab['ТИП ТС '].isin([2, '2', 'от 1,5 до 3 тн ', 'от 1,5 до 3 тн ', 'от 1,5 до 3 тн', '2 тн']), 'TS type']=2

In [231]:
data_tab.loc[data_tab['ТИП ТС '].isin([3, '3', 'от 3,1 до 5 тн', 'от 3,1 до 5 тн ', 'от 3,1 до 5', 'от 3,1 дор 5 тн ', 'от 3тн до 5 тн', 'от 3,1 до 5 ', '5 тн']), 'TS type']=3

In [232]:
data_tab.loc[data_tab['ТИП ТС '].isin([4, '4', 'от5,1 до 10тн ', 'от 5,1 до 10 тн ', 'от  5,1 до 10 тн']), 'TS type']=4

In [233]:
data_tab.loc[data_tab['ТИП ТС '].isin([5, '5', 'от 10,1 до 20 тн ', 'от 10,1 до 20тн ', 'от  10,1 до 20 тн ', 'от 10,1 до 20 тн', 'от 10,1 до 20  тн ', '5 тип']), 'TS type']=5

In [234]:
data_tab.loc[data_tab['ТИП ТС '].isin([6, '6', '20тн', '20 тн', '20т', 'до 20 тн', 'до 20тн']), 'TS type']=6

In [235]:
data_tab.loc[data_tab['ТИП ТС '].isin([7, '7', 'до 30 тн', 'до 30 тн ', 'до 30тн ', 'до 30тн ', 'трал(до 30т)']), 'TS type']=7

In [236]:
data_tab.loc[data_tab['ТИП ТС '].str.contains(r"не[а]*габарит", na=False, flags=re.IGNORECASE), 'TS type']='негабарит'

In [237]:
dict_TS={1:[3, 1.8, 1.5], 2:[3.5, 2, 2], 3:[4.5, 2, 2.2], 4:[6, 2.2, 2.4], 5:[13.6, 2.4, 2.5], 6:[15, 2.5, 3], 7:[16, 2.5, 3]}

# Этап 3. Распознавание габаритов

Следующая часть кода позволяет распознать габариты с помощью регулярных выражений и записать их в структурированный список

In [238]:
res = data_tab['Габариты грузовых мест (Д*Ш*В), м'].str.findall(r"([0-9]+[,.]*[0-9]*)[ ]*[см]*[ ]*[/*хxХх×X][ ]*([0-9]+[,.]*[0-9]*)[ ]*[см]*[ ]*[/*хxХх×X][ ]*([0-9]+[,.]*[0-9]*)[ ]*([см|мм|м]*)(?:(?:[^\w,]*(\d+)[\s]*(шт|ящ|короб))|(?:[-][^\w,]*(\d+)[\s]*(мест))?)", flags=re.IGNORECASE)

In [239]:
res[res.str.len() == 0]=data_tab['Габариты грузовых мест (Д*Ш*В), м'].str.findall(r'[дL]{1}[\D]+([0-9]+[,.]*[0-9]*)[\D]+[шB]{1}[\D]+([0-9]+[,.]*[0-9]*)[\D]+[вH]{1}[\D]+([0-9]+[,.]*[0-9]*)[ ]*([см|мм|м]*)(?:[^\w,]*(\d+)[\s]*(шт|ящ|короб))?', flags=re.IGNORECASE)

In [240]:
res[res.str.len() == 0]=data_tab['Габариты грузовых мест (Д*Ш*В), м'].str.findall(r'([0-9]+[,.]*[0-9]*)[ ]*[см]*[ ]*[/*хxХх×X][ ]*([0-9]+[,.]*[0-9]*)[ ]*([см|мм|м]*)[^ /*хxХ]*(?:[^\w,]*(\d+)[\s]*(шт|ящ|короб))?', flags=re.IGNORECASE)

In [241]:
data_tab['dimensions']=res

In [242]:
# Доля распознанных габаритов
(len(res)-len(data_tab[data_tab['dimensions'].str.len() == 0]))/len(res)

1.0

In [243]:
data_tab['dimensions']

0                          [(0,8, 0,5, 0,8, , , , , )]
1    [(200, 200, 570,, , , , , ), (900, 870, 650, ,...
2                         [(1.2, 0.8, 0,66, , , , , )]
3                           [(36, 19, 14, см, , , , )]
4    [(3280, 1660, 1320, мм, , , , ), (1990, 1970, ...
Name: dimensions, dtype: object

In [244]:
# Приведение списков к общему виду
for i in range(data_tab.shape[0]):
    if type(data_tab.loc[i, 'dimensions'])==list:
        data_tab.at[i, 'dimensions'] = list(map(list, data_tab.at[i, 'dimensions']))
        for j in range(len(data_tab.at[i, 'dimensions'])):
            if len((data_tab.at[i, 'dimensions'][j]))==8:
                data_tab.at[i, 'dimensions'][j].reverse() 
                data_tab.at[i, 'dimensions'][j].remove('')
                data_tab.at[i, 'dimensions'][j].remove('')
                data_tab.at[i, 'dimensions'][j].reverse() 

In [245]:
# Преобразование значений габаритов, в которых явно указана единица измерения, в единую размерность (в см)
for i in range(data_tab.shape[0]):
    l=data_tab.loc[i, 'dimensions']
    if type(l)==list:
        for j in range(len(l)):
            for k in range(3):
                if type(l[j][k])==str and len(l[j])==6:
                    if l[j][k]=='':
                        l[j][k]=np.nan
                    elif type(l[j][k])==str:
                        l[j][k]=float(l[j][k].replace(',', '.').replace('..', ''))
            for k in range(2):
                if type(l[j][k])==str and len(l[j])==5:
                    if l[j][k]=='':
                        l[j][k]=np.nan
                    elif type(l[j][k])==str:
                        l[j][k]=float(l[j][k].replace(',', '.').replace('..', ''))
            if l[j][3]=='м' and len(l[j])==6:
                l[j][0]=round(l[j][0]*100, 10)
                l[j][1]=round(l[j][1]*100, 10)
                l[j][2]=round(l[j][2]*100, 10)
                l[j][3]='см'
            elif l[j][3]=='мм' and len(l[j])==6:
                l[j][0]=round(l[j][0]*0.1, 10)
                l[j][1]=round(l[j][1]*0.1, 10)
                l[j][2]=round(l[j][2]*0.1, 10)
                l[j][3]='см'
            elif l[j][2]=='мм' and len(l[j])==5:
                l[j][0]=round(l[j][0]*0.1, 10)
                l[j][1]=round(l[j][1]*0.1, 10)
                l[j][2]='см'
            elif l[j][2]=='м' and len(l[j])==5:
                l[j][0]=round(l[j][0]*100, 10)
                l[j][1]=round(l[j][1]*100, 10)
                l[j][2]='см'
            data_tab.at[i, 'dimensions']=l

In [246]:
# Преобразование значений габаритов, в которых не указана единица измерения, в единую размерность (в см) в зависимости от типа трнапортного средства (для каждого типа предусмотрены свои максимально возможные габариты)
for i in range(data_tab.shape[0]):
    l=data_tab.loc[i, 'dimensions']
    if type(l)==list:
        for j in range(len(l)):
            if l[j][3]=='' and len(l[j])==6:
                if data_tab.loc[i, 'TS type'] in dict_TS.keys():
                    if l[j][0]>dict_TS[data_tab.at[i, 'TS type']][0] or l[j][1]>dict_TS[data_tab.at[i, 'TS type']][1] or l[j][2]>dict_TS[data_tab.at[i, 'TS type']][2]:
                        for k in range(3):
                            l[j][k]=round(l[j][k]*0.01, 10)
                        if l[j][0]>dict_TS[data_tab.at[i, 'TS type']][0] or l[j][1]>dict_TS[data_tab.at[i, 'TS type']][1] or l[j][2]>dict_TS[data_tab.at[i, 'TS type']][2]:
                            for k in range(3):
                                l[j][k]=round(l[j][k]*0.1, 10)
                    if l[j][0]<=dict_TS[data_tab.at[i, 'TS type']][0] and l[j][1]<=dict_TS[data_tab.at[i, 'TS type']][1] and l[j][2]<=dict_TS[data_tab.at[i, 'TS type']][2]:
                        for k in range(3):
                            l[j][k]=round(l[j][k]*100, 10)
                        data_tab.at[i, 'dimensions']=l
                else:
                    if l[j][0]>16 or l[j][1]>2.5 or l[j][2]>3:
                        for k in range(3):
                            l[j][k]=round(l[j][k]*0.01, 10)
                        if l[j][0]>=16 or l[j][1]>=2.5 or l[j][2]>=3:
                            for k in range(3):
                                l[j][k]=round(l[j][k]*0.1, 10)
                    if l[j][0]<=16 and l[j][1]<=2.5 and l[j][2]<=3:
                        for k in range(3):
                            l[j][k]=round(l[j][k]*100, 10)
                        data_tab.at[i, 'dimensions']=l
            if l[j][2]=='' and len(l[j])==5:
                if data_tab.loc[i, 'TS type'] in dict_TS.keys():
                    if l[j][0]>dict_TS[data_tab.at[i, 'TS type']][0] or l[j][1]>dict_TS[data_tab.at[i, 'TS type']][1]:
                        for k in range(2):
                            l[j][k]=round(l[j][k]*0.01, 10)
                        if l[j][0]>dict_TS[data_tab.at[i, 'TS type']][0] or l[j][1]>dict_TS[data_tab.at[i, 'TS type']][1]:
                            for k in range(2):
                                l[j][k]=round(l[j][k]*0.1, 10)
                    if l[j][0]<=dict_TS[data_tab.at[i, 'TS type']][0] and l[j][1]<=dict_TS[data_tab.at[i, 'TS type']][1]:
                        for k in range(2):
                            l[j][k]=round(l[j][k]*100, 10)
                        data_tab.at[i, 'dimensions']=l
                else:
                    if l[j][0]>16 or l[j][1]>2.5:
                        for k in range(2):
                            l[j][k]=round(l[j][k]*0.01, 10)
                        if l[j][0]>=16 or l[j][1]>=2.5:
                            for k in range(2):
                                l[j][k]=round(l[j][k]*0.1, 10)
                    if l[j][0]<=16 and l[j][1]<=2.5:
                        for k in range(2):
                            l[j][k]=round(l[j][k]*100, 10)
                        data_tab.at[i, 'dimensions']=l

# Этап 4. Распознавание количества грузовых мест

Следующая часть кода позволяет распознать количество грузовых мест с помощью регурного выражения

In [247]:
res2=data_tab['Количество грузовых мест'].astype(str).str.findall(r'(?<![хХ,\-\d(])([0-9]+)(?![-х,мх)])(?!(\d* шт)|(\d* кг)|(\d* л)|(\d*л)|(\d* бочки)|(\d*-)|(\d*х))', flags=re.I)

In [248]:
# Приведение к общему типу различных значений
for i in range(len(res2)):
    if type(res2[i])==list and len(res2[i])>0:
        res2[i]=sum([float(x[0]) for x in res2[i]])
    if type(data_tab.loc[i, 'Количество грузовых мест'])==int:
        res2[i]=data_tab.loc[i, 'Количество грузовых мест']

In [249]:
res2.head()

0    1.0
1    2.0
2    1.0
3    1.0
4    3.0
Name: Количество грузовых мест, dtype: object

In [250]:
data_tab['number_of_packages']=res2

In [251]:
# Добавление в список кол-ва грузовых мест, если оно указано и совпадает с данными габаритов грузов. Если не указано, но габариты только одни, то присваивается одно грузовое место
for i in range(data_tab.shape[0]):
    l=data_tab.loc[i, 'dimensions']
    if type(l)==list:
        for j in range(len(l)):
            if l[j][4]=='' and len(l[j])==6 and len(l)==data_tab.loc[i, 'number_of_packages']:
                l[j][4]=1
            elif l[j][4]=='' and len(l[j])==6 and len(l)==1:
                l[j][4]=data_tab.loc[i, 'number_of_packages']
            if l[j][3]=='' and len(l[j])==5 and len(l)==data_tab.loc[i, 'number_of_packages']:
                l[j][3]=1
            elif l[j][3]=='' and len(l[j])==5 and len(l)==1:
                l[j][3]=data_tab.loc[i, 'number_of_packages']
            data_tab.at[i, 'dimensions']=l   

In [252]:
#data_tab.to_excel('output.xlsx')

# Этап 5. Распознавание веса груза

Следующая часть кода позволяет распознать вес груза, привести к единой размерности

In [253]:
data_tab['Общий вес груза, кг']=data_tab['Общий вес груза, кг'].astype(str)

In [254]:
res1 = data_tab['Общий вес груза, кг'].str.findall(r"([0-9]+[,.]*[0-9]*)[ ]*(кг|тн|ТН|КГ)", flags=re.IGNORECASE)

In [255]:
for i in range(res1.shape[0]):
    if type(res1[i])==list:
        res1[i] = list(map(list, res1[i]))

In [256]:
res1 [res1.str.len() == 0] = data_tab['Общий вес груза, кг'].str.findall(r"(?<![\.х\-\d])([0-9]+[,.]*[0-9]*)(?![\w .х-])", flags=re.IGNORECASE)

In [257]:
data_tab['weight']=res1

In [258]:
for i in range(data_tab.shape[0]):
    l=data_tab.loc[i, 'weight']
    if type(l)==list and len(l)>0:
        #Одно значение
        if isinstance(l, list)==True and len(l)==1 and isinstance(l[0], list)==False:
            if type(l[0])==str:
                l[0]=float(l[0].replace(',', '.').replace('..', ''))
            if l[0]>0:
                data_tab.at[i, 'weight']=l[0]
            else:
                    data_tab.at[i, 'weight']=''
        #В списке два значения
        if isinstance(l, list)==True and isinstance(l[0], list)==False and len(l)>1:
            if len(l)>1:
                for j in range(len(l)):
                    if type(l[j])==str:
                        l[j]=float(l[j].replace(',', '.').replace('..', ''))
                summ=0
                m=[]
                for k in range(len(l)):
                    summ=summ+l[k]
                m.append(summ)
                l=m
                if l[0]>0:
                    data_tab.at[i, 'weight']=l[0]
                else:
                    data_tab.at[i, 'weight']=''
        #Много списков в одном списке
        if isinstance(l, list)==True and isinstance(l[0], list)==True:
            for j in range(len(l)):
                if type(l[j][0])==str:
                    l[j][0]=float(l[j][0].replace(',', '.').replace('..', ''))
            if len(l)>1:
                summ=0
                m=[]
                for k in range(len(l)):
                    summ=summ+l[k][0]
                m.append(summ)
                m.append(l[k][1])
                l=m
                if l[1]=='кг' or l[1]=='КГ':
                    data_tab.at[i, 'weight']=l
                elif l[1]=='тн' or l[1]=='ТН':
                    l[0]=l[0]*1000
                    l[1]='кг'
                if l[0]>0:
                    data_tab.at[i, 'weight']=l[0]
                else:
                    data_tab.at[i, 'weight']=''
            if len(l)==1:
                if type(l[0][0])==str:
                    l[0][0]=float(l[0][0].replace(',', '.').replace('..', ''))
                if l[0][1]=='кг' or l[0][1]=='КГ':
                    data_tab.at[i, 'weight']=l
                elif l[0][1]=='тн' or l[0][1]=='ТН':
                    l[0][0]=l[0][0]*1000
                    l[0][1]='кг'
                if l[0][0]>0:
                    data_tab.at[i, 'weight']=l[0][0]
                else:
                    data_tab.at[i, 'weight']=''

In [259]:
data_tab.to_excel('output.xlsx')